#Ingestion del archivo movie.csv


### Paso 1 - Leer el archivo JSON usando "DataframeReader"

In [0]:
dbutils.widgets.text("p_environment", "")
v_enviroment = dbutils.widgets.get("p_environment")

In [0]:
dbutils.widgets.text("p_file_date","2024-12-16")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
%run 
"../includes/common_functions"

In [0]:
%run
"../includes/configuration"

In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType, DateType

In [0]:
movie_genre_schema = StructType(fields=[
    StructField("movieId", IntegerType(), False),
    StructField("genreId", IntegerType(), True),
    ])

In [0]:
movie_genre_df = spark.read \
    .option("header", True) \
    .schema(movie_genre_schema) \
    .json(f"{bronze_folder_path}/{v_file_date}/movie_genre.json")

## Parte 2 : Seleccionar solo las columnas requeridas

In [0]:
from pyspark.sql.functions import col


## Paso 3. Cambiar el nombre de las columnas segun lo requerido

In [0]:
movie_rename_genre_df = movie_genre_df \
    .withColumnRenamed("movieId", "movie_id") \
    .withColumnRenamed("genreId", "genre_id") 


    

### Paso 4. Agregar la columna "Ingestion date" al dataframe

In [0]:
from pyspark.sql.functions import current_timestamp, lit
# se importan funciones de fecha del dia de ingestion en el sistema y lit que nos sirve para agragar un valor en una columna ya que si se hace sin esta funcion no sera tomada como objeto



In [0]:
movies_genre_final_df = movie_rename_genre_df \
    .withColumn("ingestion_date", current_timestamp()) \
    .withColumn("enviroment", lit("production")) \
    .withColumn("file_date", lit(v_file_date))



### Paso 5. Escribir datos en el datalake en formato "Parquet"

In [0]:
#overwrite_partition  (movies_genre_final_df, "movie_silver","movie_genre","file_date")

In [0]:
#movies_genre_final_df.write.mode("append").partitionBy("file_date").format("parquet").saveAsTable("movie_silver.movie_genre")

# Creando tabla delta y aplicando funcion merge para optimizar la carga incremental con las tablas delta lake

In [0]:
merge_key = 'tgt.movie_id = src.movie_id and tgt.genre_id = src.genre_id'
merge_delta_data(movies_genre_final_df,"/mnt/moviehistory7809/silver","movie_silver", "movie_genre", "file_date", merge_key)

In [0]:
%sql
select file_date,count(1)
from movie_silver.movie_genre
group by file_date



In [0]:
%sql
select * from movie_silver.movie_genre

In [0]:
dbutils.notebook.exit("success No 6")